In [1]:
import os
import re
import pandas as pd

### Set the folder path.

In [ ]:
# Root folder where all the "YearXXXX" folders are located
base_folder = "./CONAREME"

### Read all the CSV files.

In [3]:
# List to collect dataframes
all_dfs = []

# Walk through each subfolder
for folder_name in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder_name)
    if os.path.isdir(folder_path) and folder_name.startswith("Year"):
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.csv'):
                file_path = os.path.join(folder_path, file_name)
                df = pd.read_csv(file_path, encoding="utf-8-sig", dtype=str)

                # Add missing 'option_E' column if not present
                if 'option_E' not in df.columns:
                    df['option_E'] = pd.NA

                # Add source_folder and source_file info
                df['source_file'] = os.path.splitext(file_name)[0]
                df['source_folder'] = folder_name

                all_dfs.append(df)

# Combine all into one DataFrame
combined_df = pd.concat(all_dfs, ignore_index=True)

# Extract only the year
combined_df["year"] = combined_df["source_folder"].str.extract(r'(\d{4})').astype(int)


# Display shape or preview
print(f"Combined shape: {combined_df.shape}")
combined_df

Combined shape: (8380, 10)


,questions,option_A,option_B,option_C,option_D,correct_answer,option_E,source_file,source_folder,year
0,Varón de 65 años con antecedente de hipertensi...,Labetalol,Hidralazina,Nitroprusiato,Nitroglicerina,C,NaN,ESPECIALIADAD A,Year2025,2025
1,Mujer de 32 años asintomática que acude por ch...,Solicitar prolactina,Mamografía,Ecografía de mamas,Observación,D,NaN,ESPECIALIADAD A,Year2025,2025
2,¿Cuál es el sitio más frecuente de implantació...,Fondo uterino,Segmento inferior del útero,Cérvix uterino,Cuerno uterino,A,NaN,ESPECIALIADAD A,Year2025,2025
3,"Varón de 43 años, llevado a urgencias porque s...",Colocación de catéter central,Ventilación no invasiva,Fijación costal quirúrgica,Intubación orotraqueal,D,NaN,ESPECIALIADAD A,Year2025,2025
4,"Varón de 27 años, acude a consulta por visión ...",VI,I,III,IV,C,NaN,ESPECIALIADAD A,Year2025,2025
...,...,...,...,...,...,...,...,...,...,...
8375,Gestante de 20 años con 10 semanas de embarazo...,Epigastralgia,Meteorismo,Pérdida de peso menor al 5%,Presencia de aliento fétido,E,Alteraciones hidroelectrolíticas,GINECOLOGIA Y OBSTETRICIA,Year2020,2020
8376,"Mujer de 40 años ha tenido 4 partos eutócicos,...",Mola hidatidiforme,Hematometra,Tumor anexial,Miomatosis uterina,A,Embarazo gemelar,GINECOLOGIA Y OBSTETRICIA,Year2020,2020
8377,"Mujer de 40 años, refiere menstruaciones regul...",Biopsia en sacabocado y curetaje cervical,Histerectomía inmediata y extirpación de,Repetir el frotis de Papanicolaou y control a los,Colposcopía con biopsia,D,Dilatación y legrado fraccionado,GINECOLOGIA Y OBSTETRICIA,Year2020,2020
8378,"De los siguientes exámenes o procedimientos, ¿...",Eco transvaginal,Eco doppler transvaginal,Cultivo de secreción vaginal,Histeroscopía,E,Laparoscopia,GINECOLOGIA Y OBSTETRICIA,Year2020,2020


### Look for possible numbers which may have been treated as dates.
Then fix those errors, and again look for errors again to verify there are no more errors.

In [4]:
# --- tune this: columns you want to scan (or None for all)
COLUMNS_TO_SCAN = None  # e.g., ["option_A","option_B","option_C"]

# Month names (English + Spanish, abbr + full)
MONTHS_ABBR = r"Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec|Ene|Abr|Ago|Dic"
MONTHS_FULL = r"January|February|March|April|May|June|July|August|September|October|November|December|" \
              r"Enero|Febrero|Marzo|Abril|Mayo|Junio|Julio|Agosto|Septiembre|Octubre|Noviembre|Diciembre"

# Patterns to catch common “date-like” forms while avoiding IDs like "A-12"
PATTERNS = [
    # 1) Purely numeric: 1/2, 01/02, 1-2, 01-02, optional year 1/2/23 or 2023-01-02
    r"^\s*\d{1,2}[/-]\d{1,2}([/-]\d{2,4})?\s*$",
    r"^\s*\d{4}[/-]\d{1,2}[/-]\d{1,2}\s*$",
    # 2) Day-Month (abbr or full): 12-Jun, 12 June, 12-Jun-23, 12 June 2023
    rf"^\s*\d{{1,2}}[.\-/ ](?:{MONTHS_ABBR}|{MONTHS_FULL})(?:[.\-/ ]\d{{2,4}})?\s*$",
    # 3) Month-Day (abbr or full): Jun-12, June 12, Jun/12/2023, June-12-23
    rf"^\s*(?:{MONTHS_ABBR}|{MONTHS_FULL})[.\-/ ]\d{{1,2}}(?:[.\-/ ]\d{{2,4}})?\s*$",
]

COMBINED = re.compile("|".join(PATTERNS), flags=re.IGNORECASE)

def find_date_like(df: pd.DataFrame, columns=None) -> pd.DataFrame:
    cols = df.columns if columns is None else columns
    sdf = df[cols].astype(str)
    mask = sdf.apply(lambda col: col.str.match(COMBINED, na=False))
    return mask


# --- Detect date-like strings ---
date_like_mask = find_date_like(combined_df, columns=COLUMNS_TO_SCAN)

# --- Show counts per column ---
print("Suspicious values per column:")
print(date_like_mask.sum())

# --- Show the problematic rows ---
problem_rows = combined_df[date_like_mask.any(axis=1)]
problem_rows

Suspicious values per column:
questions          0
option_A          26
option_B          23
option_C          24
option_D          25
correct_answer     0
option_E          10
source_file        0
source_folder      0
year               0
dtype: int64


,questions,option_A,option_B,option_C,option_D,correct_answer,option_E,source_file,source_folder,year
174,"¿Cuál es la duración en minutos, de la acción ...",120-180,360-460,60-80,180-360,C,NaN,ESPECIALIADAD B,Year2025,2025
283,La indicación de ayuno preoperatorio en pacien...,8/6/04,6/3/02,8/4/02,6/4/02,D,NaN,ANESTESIOLOGIA,Year2025,2025
315,¿Cuál es el porcentaje de supervivencia a los ...,36-79,10-25,27-57,59-64,C,NaN,CIRUGIA CARDIOVASCULAR,Year2025,2025
1115,¿Cuál es el porcentaje de supervivencia a los ...,36-79,10-25,27-57,59-64,C,NaN,CIRUGIA DE TORAX Y CARDIOVASCULAR,Year2025,2025
1526,"Mujer de 60 años, desde hace 1 semana presenta...",Más de 40,5- 10,menos de 5,30-40,E,10- 20,Especialidad Prueba A,Year2022,2022
1964,Varón 55 años intervenido de cirugía cardíaca;...,5-10,10-15,60-90,90-120,E,15-60,Cirugía Cardiovascular,Year2022,2022
1985,"El estudio en un corazón infartado, muestra bo...",10-14,7-9,3-6,1-2,A,15-19,Cirugía Cardiovascular,Year2022,2022
2473,Varón de 50 años que sale temprano a realizar ...,15/2,30/1,30/2,15/1,C,20/2,Anestesiología,Year2022,2022
2819,¿Cuál es el nivel de BIS para profundidad anes...,75-80,30-35,85-90,40-60,D,NaN,ANESTESIOLOGIA,Year2023,2023
2899,En anestesia general y/o hipnosis profunda. ¿C...,13-19,20-26,37-46,47-56,B,NaN,ANESTESIOLOGIA,Year2023,2023


In [5]:
column_maps = {
    "option_A": {
        "15-Oct": "10 - 15",
        "6-Apr": "4 - 6"
    },

    "option_B": {
        "10-May": "5 - 10",
        "10-Aug": "8 - 10"
    },

    "option_C": {
        "15-Dec": "12 - 15"
    },

    "option_D": {
        "2-Jan": "1 - 2",
        "Oct-90": "10 / 90",
        "Oct-50": "10-50"
    },

    "option_E": {
        "3-Feb": "2 - 3"
    }
}

for col, mapping in column_maps.items():
    if col in combined_df.columns:
        combined_df[col] = combined_df[col].replace(mapping)


In [6]:
# --- Detect date-like strings ---
date_like_mask = find_date_like(combined_df, columns=COLUMNS_TO_SCAN)

# --- Show counts per column ---
print("Suspicious values per column:")
print(date_like_mask.sum())

# --- Show the problematic rows ---
problem_rows = combined_df[date_like_mask.any(axis=1)]
problem_rows

Suspicious values per column:
questions          0
option_A          24
option_B          21
option_C          23
option_D          23
correct_answer     0
option_E           9
source_file        0
source_folder      0
year               0
dtype: int64


,questions,option_A,option_B,option_C,option_D,correct_answer,option_E,source_file,source_folder,year
174,"¿Cuál es la duración en minutos, de la acción ...",120-180,360-460,60-80,180-360,C,NaN,ESPECIALIADAD B,Year2025,2025
283,La indicación de ayuno preoperatorio en pacien...,8/6/04,6/3/02,8/4/02,6/4/02,D,NaN,ANESTESIOLOGIA,Year2025,2025
315,¿Cuál es el porcentaje de supervivencia a los ...,36-79,10-25,27-57,59-64,C,NaN,CIRUGIA CARDIOVASCULAR,Year2025,2025
1115,¿Cuál es el porcentaje de supervivencia a los ...,36-79,10-25,27-57,59-64,C,NaN,CIRUGIA DE TORAX Y CARDIOVASCULAR,Year2025,2025
1526,"Mujer de 60 años, desde hace 1 semana presenta...",Más de 40,5- 10,menos de 5,30-40,E,10- 20,Especialidad Prueba A,Year2022,2022
1964,Varón 55 años intervenido de cirugía cardíaca;...,5-10,10-15,60-90,90-120,E,15-60,Cirugía Cardiovascular,Year2022,2022
1985,"El estudio en un corazón infartado, muestra bo...",10-14,7-9,3-6,1-2,A,15-19,Cirugía Cardiovascular,Year2022,2022
2473,Varón de 50 años que sale temprano a realizar ...,15/2,30/1,30/2,15/1,C,20/2,Anestesiología,Year2022,2022
2819,¿Cuál es el nivel de BIS para profundidad anes...,75-80,30-35,85-90,40-60,D,NaN,ANESTESIOLOGIA,Year2023,2023
2899,En anestesia general y/o hipnosis profunda. ¿C...,13-19,20-26,37-46,47-56,B,NaN,ANESTESIOLOGIA,Year2023,2023


### Rename the speciality names into consistent and grouped categories.

In [7]:
# Standardize the name of the specialities
mapping = {
    'Especialidad Prueba B': 'Prueba B',
    'Especialidad Prueba A': 'Prueba A',
    'Ginecología y Obstetricia': 'Ginecología y Obstetricia',
    'Radiología': 'Radiología',
    'Neurocirugía': 'Neurocirugía',
    'Cirugía Cardiovascular': 'Cirugía de Tórax y Cardiovascular',
    'Urología': 'Urología',
    'Oftalmología': 'Oftalmología',
    'Psiquiatría': 'Psiquiatría',
    'Pediatría': 'Pediatría',
    'Anestesiología': 'Anestesiología',
    'Cirugía General': 'Cirugía General',
    'CIRUGÍA DE TORÁX Y CARDIOVASCULAR': 'Cirugía de Tórax y Cardiovascular',
    'RADIOLOGÍA': 'Radiología',
    'ANESTESIOLOGIA': 'Anestesiología',
    'GINECOLOGIA': 'Ginecología y Obstetricia',
    'CIRUGIA GENERAL': 'Cirugía General',
    'NEUROCIRUGIA': 'Neurocirugía',
    'PRUEBA A': 'Prueba A',
    'UROLOGÍA': 'Urología',
    'PRUEBA B': 'Prueba B',
    'OFTALMOLOGÍA': 'Oftalmología',
    'PSIQUIATRÍA': 'Psiquiatría',
    'PEDIATRÍA': 'Pediatría',
    'ANATOMIA PATOLOGICA': 'Anatomía Patológica & Patología',
    'ESPECIALIADAD A': 'Prueba A',
    'ESPECIALIADAD B': 'Prueba B',
    'ANATOMÍA PATOLÓGICA': 'Anatomía Patológica & Patología',
    'GINECOLOGÍA': 'Ginecología y Obstetricia',
    'NEUROCIRUGÍA': 'Neurocirugía',
    'CIRUGÍA DE TORAX': 'Cirugía de Tórax y Cardiovascular',
    'CIRUGÍA': 'Cirugía General',
    'CIRUGIA': 'Cirugía General',
    'OFTALMOLOGIA': 'Oftalmología',
    'CIRUGÍA CARDIOVASCULAR': 'Cirugía de Tórax y Cardiovascular',
    'PSIQUIATRIA': 'Psiquiatría',
    'PEDIATRIA': 'Pediatría',
    'PATOLOGIA': 'Anatomía Patológica & Patología',
    'UROLOGIA': 'Urología',
    'RADIOLOGIA': 'Radiología',
    'GINECOLOGIA Y OBSTETRICIA': 'Ginecología y Obstetricia',
    'CIRUGIA DE TORAX Y CARDIOVASCULAR': 'Cirugía de Tórax y Cardiovascular',
    'CIRUGIA CARDIOVASCULAR': 'Cirugía de Tórax y Cardiovascular'
}
combined_df['source_file'] = combined_df['source_file'].replace(mapping)
combined_df['source_file'].unique()

array(['Prueba A', 'Prueba B', 'Anestesiología',
       'Cirugía de Tórax y Cardiovascular', 'Cirugía General',
       'Neurocirugía', 'Oftalmología', 'Psiquiatría', 'Urología',
       'Radiología', 'Pediatría', 'Anatomía Patológica & Patología',
       'Ginecología y Obstetricia'], dtype=object)

In [8]:
combined_df['source_file'].value_counts()

source_file
Anestesiología                       700
Cirugía General                      700
Oftalmología                         700
Psiquiatría                          700
Pediatría                            700
Ginecología y Obstetricia            700
Prueba A                              690
Prueba B                              690
Cirugía de Tórax y Cardiovascular    600
Urología                             600
Radiología                           600
Anatomía Patológica & Patología     600
Neurocirugía                         400
Name: count, dtype: int64

### Remember that some tests do not have "option_E" (i.e., only four options). For those cases, we added "optinon_E" as NA and now we're chaging that to "Ninguna de las Anteriores" (Spanish for "none of the above").

In [9]:
# For tests which did not have Option E, fill in.
combined_df['option_E'] = combined_df['option_E'].fillna('Ninguna de las anteriores')
combined_df['option_E'].isna().sum()   # Now there should 0 NAs in this column.

np.int64(0)

### In the file "manual_changes" we kept track of how many "correct_answer" we needed to manually update because the extraction got them wrong.
Compute the proportion of manual changes over all questions.

In [10]:
# from file "manual_changes", 16 correct_answers were manually changed.
((16 / combined_df.shape[0])*100)   # -> The cases we manually corrected were <1% of all questions.

0.1909307875894988

### Save the datasets.

In [ ]:
# Save as CSV and Pickle
combined_df.to_csv("./01.Datasets/combined_exam_dataset.csv", index = False)
combined_df.to_pickle("./01.Datasets/combined_exam_dataset.pkl")